In [ ]:
#К сожалению, у нас произошли проблемы с импортом данных в PostgreSQL. Поэтому используем данный скрипт. 
#Загрузите базу risks из файла (некоторые пункты скрипта на ней уже могут быть выполнены), разверните на сервере или можно создать новую базу и загрузить мастером импорта таблицы в базу из файлов csv. В базе должны быть 4 таблицы base1, base2, instrs, bond_description. 
#Далее следует запустить скрипт, который создаст 3 связанные ключами таблицы. Части скрипта нужно запускать отдельно. Следует убедиться, что скрипты запускаются в нужной базе risks.
#скрипт 1. объединение таблиц торгов
SELECT * INTO [risks].[dbo].[base_prices] FROM [risks].[dbo].[base1]
UNION ALL
SELECT * FROM [risks].[dbo].[base2];
DROP TABLE IF EXIST [risks].[dbo].[base1];
DROP TABLE IF EXIST [risks].[dbo].[base2];
GO

#изменение форматов данных для установки ключей
ALTER TABLE [risks].[dbo].[base_prices] ALTER COLUMN ID int NOT NULL;
ALTER TABLE [risks].[dbo].[base_prices] ALTER COLUMN ISIN varchar(50) NOT NULL;
ALTER TABLE [risks].[dbo].[bond_discription] ALTER COLUMN [ISIN, RegCode, NRDCode] varchar(50) NOT NULL;
ALTER TABLE [risks].[dbo].[instrs] ALTER COLUMN ID int NOT NULL;
GO

#установка первичных и внешних ключей. Если появляется ошибка, проверить в базе, установлены ли ключи 
ALTER TABLE [risks].[dbo].[instrs] ADD PRIMARY KEY (ID);
ALTER TABLE [risks].[dbo].[bond_discription] ADD PRIMARY KEY ([ISIN, RegCode, NRDCode]);
ALTER TABLE [risks].[dbo].[base_prices] ADD CONSTRAINT FK_base1 FOREIGN KEY (ID) REFERENCES [HW].[dbo].[instrs] (ID); 
GO 

 SELECT [risks].[dbo].[base_prices].[ID] INTO [risks].[dbo].[unknownISINs_ID]
 FROM [risks].[dbo].[base_prices] LEFT JOIN [risks].[dbo].[bond_discription] 
	ON [base_prices].[ISIN] = [bond_discription].[ISIN, RegCode, NRDCode]
	WHERE [bond_discription].[ISIN, RegCode, NRDCode] IS NULL;
 SELECT * INTO [risks].[dbo].[uknownISINs_prices]
    FROM [risks].[dbo].[base_prices]  
		WHERE [ID] IN (SELECT [ID] FROM [risks].[dbo].[uknownISINs_ID]) ;
	GO
 DELETE FROM [risks].[dbo].[base_prices]  
		WHERE [ID] IN (SELECT [ID] FROM [risks].[dbo].[unknownISINs_ID]) ;
	GO

ALTER TABLE [risks].[dbo].[base_prices] ADD CONSTRAINT FK_base2 FOREIGN KEY (ISIN)     
    REFERENCES [risks].[dbo].[bond_discription] ([ISIN, RegCode, NRDCode]);    
GO